# Import & Load Childhood data (Under-One Year)

### Config

In [1]:
import sys
import configparser
config = configparser.ConfigParser()
config.read_file(open('../../../settings.ini'))

sys.path.insert(0, config.get('PATHS','libs_path'))
engine_path = config.get('DATABASE','engine_path')

### Main params

In [2]:
remote_path = ''
local_path = '../data/'

In [3]:
import postgres #from local file postgres.py
from commons import inline_table_xml, download_file, download_zip_file, extract_zip_file #from local file commons.py

import json
import pandas as pd
from sqlalchemy import create_engine

### Load file

In [4]:
#Open unzipped file
df = pd.read_csv(local_path+'under_one.csv',delimiter=",")
df = df[['year','comuna_id','rate_comuna','rate_region','rate_country','count', 'age_range_id']]
df = df.astype({'year':'int','comuna_id':'int','rate_comuna':'float','rate_region':'float','rate_country':'float', 'age_range_id':'int'})
list(df)

['year',
 'comuna_id',
 'rate_comuna',
 'rate_region',
 'rate_country',
 'count',
 'age_range_id']

### Ingest

In [5]:
engine = create_engine(engine_path)
db = postgres.PostgresDriver(engine)
db.to_sql(df, 'childhood', 'fact_mortality_under_one')

DROP TABLE IF EXISTS childhood.fact_mortality_under_one;
CREATE TABLE "childhood"."fact_mortality_under_one" (
"year" INTEGER,
  "comuna_id" INTEGER,
  "rate_comuna" REAL,
  "rate_region" REAL,
  "rate_country" REAL,
  "count" INTEGER,
  "age_range_id" INTEGER
)
COPY "childhood"."fact_mortality_under_one" ("year","comuna_id","rate_comuna","rate_region","rate_country","count","age_range_id") FROM STDIN WITH CSV HEADER DELIMITER ',';


### Add Indices to foreign keys

In [6]:
engine.execute("""
CREATE INDEX fact_mortality_under_one_comuna_id_index 
ON childhood.fact_mortality_under_one (comuna_id)
""")

engine.execute("""
CREATE INDEX fact_mortality_under_one_year_index 
ON childhood.fact_mortality_under_one ("year");
""")

### Relation tables

```xml
<InlineTable alias="sex">
  <ColumnDefs>
    <ColumnDef name="id" type="Numeric" />
    <ColumnDef name="description" type="String" />
    <ColumnDef name="es_description" type="String" />
  </ColumnDefs>
  <Rows>
    <Row>
      <Value column="id">0</Value>
      <Value column="description">No informa</Value>
      <Value column="es_description">No informa</Value>
    </Row>
  <Row>
      <Value column="id">1</Value>
      <Value column="description">Women</Value>
      <Value column="es_description">Mujer</Value>
    </Row>
  <Row>
      <Value column="id">2</Value>
      <Value column="description">Men</Value>
      <Value column="es_description">Hombre</Value>
    </Row>
  </Rows>
</InlineTable>
```

In [7]:
d2 = download_file(remote_path,local_path,'age_range_id.csv')
print (inline_table_xml(d2, 'en', 'id', 'es'))

Already downloaded. Using: ../data/age_range_id.csv

<InlineTable alias="en">
  <ColumnDefs>
    <ColumnDef name="id" type="Numeric" />
    <ColumnDef name="description" type="String" />
    <ColumnDef name="es_description" type="String" />
  </ColumnDefs>
  <Rows>
    <Row>
      <Value column="id">1</Value>
      <Value column="description">Defunciones menores 7 días</Value>
      <Value column="es_description">Defunciones menores 7 días</Value>
    </Row>
  <Row>
      <Value column="id">2</Value>
      <Value column="description">Defunciones menores 28 días</Value>
      <Value column="es_description">Defunciones menores 28 días</Value>
    </Row>
  <Row>
      <Value column="id">3</Value>
      <Value column="description">Defunciones 28 días a 11 meses</Value>
      <Value column="es_description">Defunciones 28 días a 11 meses</Value>
    </Row>
  <Row>
      <Value column="id">4</Value>
      <Value column="description">Defunciones menores de 1 año</Value>
      <Value column="es